In [ ]:
import re

# Part 1

In [ ]:
def get_sequence(filename):
    sequence = []
    with open(filename) as file:
        for line in file:
            coordinates = re.findall(r"(-?\d+)", line)
            state = line.startswith("on")
            sequence.append((state, *map(int, coordinates)))
    return sequence

In [ ]:
sequence = get_sequence("day22.input")

In [ ]:
import itertools

grid = dict()
for state, x_min, x_max, y_min, y_max, z_min, z_max in sequence:
    if min(x_min, y_min, z_min) < -50 or max(x_max, y_max, z_max) > 50:
        continue
    for x, y, z in itertools.product(
                         range(x_min, x_max + 1),
                         range(y_min, y_max + 1),
                         range(z_min, z_max + 1)):
        grid[(x, y, z)] = state

In [ ]:
sum(grid.values())

# Part 2

We cannot loop over all those cubes, it would take too long... We need to somehow use just the intervals, and keep track of the overlapping cuboids.

Pseudo-code for solution:
- Start with an empty list `L` of cuboids
- For each new cuboid `N` in the boot sequence:
    - For each already existing cuboid `C` in `L`:
        - If there is an intersection `I` (also a cuboid) between `N` and `C`:
            - Add the intersecton `I` to `L` with value = opposite sign of `C`, after finished processing all `C`. 
    - If the value of `N` is `on`, add it to `L` with a value of `+1`
- Calculate the `sum(C.volume*C.value for C in L)`

For every `N` in the sequence, we loop over an increasingly longer `L`, so the best case (no intersections), the algorithm is $O(n^2)$. In practice, it will be slower...

In [ ]:
from dataclasses import dataclass
import math

In [ ]:
@dataclass
class Cuboid:
    ranges: tuple[int]  # x1, x2, y1, y2, z1, z2
    value: int = None

    @property
    def volume(self):
        return math.prod((b - a + 1) for a, b in self._pairwise(self.ranges))
    
    @property
    def count(self):
        return self.value*self.volume
    
    def _pairwise(self, seq):
        return zip(seq[::2], seq[1::2])

    def intersection(self, other):
        ranges = []
        for s, o in zip(self._pairwise(self.ranges), self._pairwise(other.ranges)):
            left, right = max(s[0], o[0]), min(s[1], o[1])
            if left > right:
                return
            ranges.extend((left, right))
        return Cuboid(tuple(ranges), -1)

In [ ]:
def get_cuboids(filename):
    cuboids = []
    with open(filename) as file:
        for line in file:
            coordinates = re.findall(r"(-?\d+)", line)
            action = line.split()[0]
            cuboids.append(Cuboid(tuple(map(int, coordinates)), action))
    return cuboids

In [ ]:
def solve_part2(filename):
    cuboids = []
    for new in get_cuboids(filename):
        intersections = []
        for cuboid in cuboids:
            if intersection := cuboid.intersection(new):
                intersection.value = -cuboid.value
                intersections.append(intersection)
        cuboids.extend(intersections)
        if new.value == "on":
            new.value = 1
            cuboids.append(new)
            
    return sum(cuboid.count for cuboid in cuboids)

In [ ]:
assert solve_part2("day22_example2.input") == 2758514936282235

In [ ]:
# This takes approx. 50s on my 2016 MacBook Pro

solve_part2("day22.input")